# Урок 7

## Генерация текста с помощью RNN

Поставим задачу: необходимо сгенерировать научный текст.

В качестве датасета возьмем [выгрузку статей из Arxiv](https://www.kaggle.com/datasets/Cornell-University/arxiv). Используем версию 175 (4.13 GB).

Метрику качества выберем "на глаз".

Модель возьмем RNN, функция потерь будет кросс-энтропия, оптимизатор Adam.

Модель будет предсказывать следующий токен. Кросс-энтропию будем считать над вероятностью реального токена.

In [1]:
from random import sample

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import wandb
from torch.optim import Adam

### Токенизация

В качестве токена в этой задаче будем брать **один символ**.

In [2]:
# BOS — символ начала текста, EOS — символ конца текста
BOS, EOS = " ", "\n"

lines = []
# возьмем только каждую 10-ую строку в финальный датасет
with open("arxiv-metadata-oai-snapshot.json", "r") as f:
    for i, one_line in enumerate(tqdm.tqdm(f.readlines())):
        if i % 10 == 0:
            lines.append(one_line)
with open("small-data.json", "w") as f:
    f.writelines(lines)

data = pd.read_json("small-data.json", lines=True)
lines = (
    data.apply(lambda row: (row["title"] + " ; " + row["abstract"])[:512], axis=1)
    .apply(lambda line: BOS + line.replace(EOS, " ") + EOS)
    .tolist()
)

100%|██████████| 2459562/2459562 [00:00<00:00, 5483201.82it/s]


In [3]:
lines[0]

' Calculation of prompt diphoton production cross sections at Tevatron and   LHC energies ;   A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region o\n'

In [4]:
tokens = {one_char for one_line in lines for one_char in one_line}

tokens = sorted(tokens)
n_tokens = len(tokens)
print("n_tokens = ", n_tokens)

n_tokens =  97


In [5]:
token_to_id = {x: i for i, x in enumerate(tokens)}

### Паддинги

In [6]:
def to_tensor(
    lines: list[str],
    max_len: int | None = None,
    pad: str = token_to_id[EOS],
    dtype=torch.int64,
):
    max_len = max_len or max(map(len, lines))
    lines_ix = torch.full([len(lines), max_len], pad, dtype=dtype)
    for i in range(len(lines)):
        line_ix = [token_to_id[x] for x in lines[i][:max_len]]
        lines_ix[i, : len(line_ix)] = torch.tensor(line_ix)
    return lines_ix


print(to_tensor([" abc\n", " abacaba\n", " abc1234567890\n"]))

tensor([[ 1, 66, 67, 68,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 66, 67, 66, 68, 66, 67, 66,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 66, 67, 68, 18, 19, 20, 21, 22, 23, 24, 25, 26, 17,  0]])


In [7]:
def compute_mask(input_ix, eos_ix=token_to_id[EOS]):
    return F.pad(
        torch.cumsum(input_ix == eos_ix, dim=-1)[..., :-1] < 1,
        pad=(1, 0, 0, 0),
        value=True,
    )


print(compute_mask(to_tensor([" hello there\n"], max_len=15)))
print(compute_mask(to_tensor([" hello there\n"], max_len=18)))

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False, False]])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False, False, False, False, False]])


### Модель

In [8]:
ref_rnn = nn.RNN(10, 32)
print(ref_rnn.weight_hh_l0.shape)
print(ref_rnn.weight_ih_l0.shape)
print(ref_rnn.bias_hh_l0.shape)
print(ref_rnn.bias_ih_l0.shape)

torch.Size([32, 32])
torch.Size([32, 10])
torch.Size([32])
torch.Size([32])


In [9]:
class MyRnnLayer(nn.Module):
    # tanh(W_x * x + W_h * h + b_x + b_h) = tanh(W_x * x + b_x     +     W_h * h + b_h)
    def __init__(self, in_dim: int, hid_size: int):
        super().__init__()
        self.hid_size = hid_size
        self.input_linear_layer = nn.Linear(
            in_features=in_dim, out_features=hid_size, bias=True
        )
        self.hidden_state_linear_layer = nn.Linear(
            in_features=hid_size, out_features=hid_size, bias=True
        )
        self.activation = nn.Tanh()

    def forward(self, x: torch.Tensor, state: torch.Tensor | None = None):
        assert x.ndim == 3  # (bs, n_tokens, token_dim)
        if state is None:
            state = torch.zeros((x.shape[0], self.hid_size), device=x.device)
        states = [state]
        for i in range(x.shape[1]):
            states.append(
                self.activation(
                    self.input_linear_layer(x[:, i, :])
                    + self.hidden_state_linear_layer(states[-1])
                )
            )
        return torch.stack(states[1:]).permute((1, 0, 2)), states[-1][None, ...]


my_rnn = MyRnnLayer(10, 32)
test_input = torch.randn((5, 12, 10))
test_output = my_rnn(test_input)
test_output[0].shape, test_output[1].shape

(torch.Size([5, 12, 32]), torch.Size([1, 5, 32]))

In [10]:
from torch.testing import assert_close

# Протестируем: наш слой должен давать те же результаты, что слой в pytorch
ref_rnn = nn.RNN(10, 32, batch_first=True)
with torch.no_grad():
    my_rnn.hidden_state_linear_layer.weight.copy_(ref_rnn.weight_hh_l0.clone())
    my_rnn.input_linear_layer.weight.copy_(ref_rnn.weight_ih_l0.clone())
    my_rnn.hidden_state_linear_layer.bias.copy_(ref_rnn.bias_hh_l0.clone())
    my_rnn.input_linear_layer.bias.copy_(ref_rnn.bias_ih_l0.clone())
    for i in (0, 1):
        actual = my_rnn(test_input)[i]
        expected = ref_rnn(test_input)[i]
        assert_close(actual, expected)

In [ ]:
# 3 токена
# [0, 1, 2]
# размерность эмбеддинга 2
# 3x2 — матрица эмбеддингов

[
    [a, b],
    [c, d],
    [e, f],
]

# embbeding[0] --> [a, b]
# embbeding[2] --> [e, f]

In [11]:
class RNNLanguageModel(nn.Module):
    def __init__(
        self, n_tokens: int = n_tokens, emb_size: int = 16, hid_size: int = 256
    ):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings=n_tokens, embedding_dim=emb_size)
        self.rnn = MyRnnLayer(emb_size, hid_size)
        # То же самое, что:
        # self.rnn = nn.RNN(emb_size, hid_size, batch_first=True)
        self.linear = nn.Linear(in_features=hid_size, out_features=n_tokens)

    def forward(self, input_ix):
        # input_ix -> (bs, n_tokens) = (1, 4)
        rv = self.emb(input_ix)  # (1, 4, 16)
        rv = self.rnn(rv)[0]  # (1, 4, 256)
        rv = self.linear(rv)  # (1, 4, 256) @ (256, 97) = (1, 4, 97)
        return rv


model = RNNLanguageModel()
model(to_tensor(["in this article we"])).shape

torch.Size([1, 18, 97])

### Функция потерь и генерация текста

#### Функция потерь
Модель выдает вероятности каждого токена из словаря для каждого слова.
Будем учить модель предсказывать следующий токен при условии всех предыдущих.

![pic](./pic.jpg)

Формула функции потерь:
$$
L = - \cfrac{1}{N} \sum_{i=1}^N \ln p(x_t^{(i)} | x_{t-1}^{(i)}, \dots, x_1^{(i)})
$$
где $N$ — размер батча.

#### Генерация
Поскольку модель выдает вектор вероятностей каждого токена в предложении, то для генерации одного текста поступим так:
1. Берем вектор `(batch_size = 1, n_words, emb_dim)` из выхода модели, берем последнюю координату из `n_words`.
2. Из полученного вектора `(1, emb_dim)` вероятностей решаем, как получить следующий токен.

Существует два варианта того, как можно получить следующий токен:
1. Отобрать тот, у кого самая большая вероятность — это **жадный выбор** (greedy sampling). Просто берем `argmax`.
2. Взять случайный с учетом вероятностей.
Модель выдает логиты — к ним применим softmax и получим вероятности.
Пример сэмплирования: в векторе `[0.1, 0.3, 0.6]` первая координата будет выбрана с вероятностью 10%, вторая — с 30%, третья — с 60%.
Это называется **случайное сэмплирование с учетом вероятностей**.
3. Взять идею из п.2, но перед сэмплированием перевзвесить все логиты:
$$
p(l_i) = \cfrac{\exp(l_i / \tau)}{\sum_{i=1}^D \exp(l_i / \tau)}
$$
где $D$ — мощность словаря, число уникальных токенов, $l_i$ — логит для $i$-го токена, $\tau$ — некоторый параметр, называемый **температурой**.

Это называется **сэмплирование с температурой**.
Варьируя температуру, можно либо перейти в жадный выбор, либо в равновероятный выбор токенов.

Мы будем использовать п.1 и п.3 для сэмплирования.

In [12]:
def compute_loss(model: nn.Module, input_ix: torch.Tensor, device: str = "cpu"):
    input_ix = torch.as_tensor(input_ix, dtype=torch.int64)
    input_ix = input_ix.to(device)

    # (bs, sentence_length)
    logits = model(input_ix[:, :-1])  # (bs, sentence_length, 97)
    reference_answers = input_ix[:, 1:]  # (bs, sentence_length) [32, 13, 28, 1]
    rv = torch.softmax(logits, 2)    # (bs, sentence_length, 97) --> (bs, sentence_length)
    # (bs, sentence_length, 97)
    # (bs, [0], [32])
    # (bs, [1], [13])
    # (bs, [2], [28])
    # (bs, [3], [1])
    # [:, :, 0]  # (a, b, c) -> (a, b)
    rv = torch.gather(rv, 2, reference_answers[:, :, None]).squeeze(2)  # (bs, sentence_length)
    rv = torch.log(rv)
    # Потери считаем только для реальных токенов, паддинги не включаем.
    # Для этого умножим на маску — она равна 1 для реальных токенов и 0 для паддингов.
    rv = rv * compute_mask(input_ix)[:, 1:]
    return -torch.sum(rv) / input_ix.shape[0]


def generate(
    model: nn.Module,
    prefix: str = BOS,
    temperature: float = 1.0,
    max_len: int = 100,
    device: str = "cpu",
):
    with torch.no_grad():
        while True:
            probs = (
                torch.softmax(model(to_tensor([prefix]).to(device))[0, -1], dim=-1)
                .cpu()
                .numpy()
            )
            if temperature == 0:
                next_token = tokens[np.argmax(probs)]
            else:
                probs = np.array([p ** (1.0 / temperature) for p in probs])
                probs /= sum(probs)
                next_token = np.random.choice(tokens, p=probs)

            prefix += next_token
            if next_token == EOS or len(prefix) > max_len:
                break
    return prefix

### Обучение и результаты

In [13]:
def train_loop(model: nn.Module, train_lines: list[str], device: str = "cpu"):
    run = wandb.init(project="start-dl--lesson-7")
    clip_norm = 1e5
    batch_size = 64
    opt = Adam(model.parameters())
    train_history = []
    model.to(device)
    text_table = wandb.Table(columns=["iteration", "text"])
    for i in tqdm.trange(len(train_history), 3000):
        batch = to_tensor(sample(train_lines, batch_size)).to(device)
        loss_i = compute_loss(model, batch, device=device)

        opt.zero_grad()
        loss_i.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
        opt.step()

        train_history.append((i, float(loss_i)))
        wandb.log({"loss": loss_i.detach().cpu().item()})

        if (i + 1) % 50 == 0:
            for _ in range(3):
                example = generate(model, temperature=0.5, device=device)
                text_table.add_data(i, example)
    run.log({"train-samples": text_table})


device = "cuda" if torch.cuda.is_available() else "cpu"
train_loop(model, lines, device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: alekseik1. Use `wandb login --relogin` to force relogin


/home/aleksei/.cache/pypoetry/virtualenvs/start-dl-fEQaQ9Q8-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 3000/3000 [07:55<00:00,  6.31it/s]


In [14]:
for _ in range(5):
    print('###')
    print(generate(model, temperature=0.5, max_len=200, device=device))

###
 On the controlled recently, an explained by the construction of the spin complex ;   The computations are been scale from the same of stable structure of $\mathbb{Q}$ of structure of the magnetic fiel
###
 Results of the singlet and analysis of the   interactions by a simulations of a function of a crystalling a first between the recent all several original models with a second problem of the discrete t
###
 Structure with the properties of the research in the controlled to the dependent of the properties of the effective excitations of the magnetory (observing the contains the problem is a multiple and d
###
 The condition of a structures of the superconductivity is investigated by the calculations of this studied the High measurements ;   We study the constraints in the sample of the statistics of the top
###
 A Sells between the singular core interaction for a consistent and superconducting models are proposed in the complexity of the mass model to prove dependent structure of 

Получилось вполне правдоподобно.

В качестве упражнения попробуйте подвигать температуру:
1. Слишком большая температура сделает токены более равновероятными. Из-за этого слова начнут превращаться в кашу.
2. Слишком маленькая температура будет делать обратное — заострять пики распределения. Из-за этого сэмплирование превратится в жадное.

## Резюме

1. Узнали, как решать задачу генерации текста с помощью RNN в PyTorch.
2. Посмотрели, как формализовать задачу предсказания токена.
3. Познакомились со стратегиями генерации токенов:
    - жадное сэмплирование;
    - случайное сэмплирование с температурой.